## Capstone Project - The Battle of Neighborhoods (Week 2)


## Background
Being one of the world's most flourishing regions thanks to its bustling economic environment, New York is an ideal candidate for ambitious entrepreneurs. Data shows that among New Yorkers are some of the most wealthy businessmen of the world; this is reflected, however, by an increase in the cost of living, in the form of expensive rents and steep taxes. While this does not discourage new businesses from emerging, it makes choosing the right setting a difficult matter.

## Problem
Possibly the biggest challenge for any New York City business is going to be the wide array of competition. With more than 8 million New Yorkers, the market tends to be over-saturated in almost every industry. Businesses now have to go farther than just offering a service and focus on well thought out marketing strategies to grow. With Queens having one of the most diverse places in the nation. Half of the neighborhood’s residents speak Spanish. Others speak Chinese, Urdu, Hindi, Russian, Portuguese, Greek or Korean. Altogether, the neighborhood is said to be the home of 167 languages. It will be a challenges on what business to start and offer these culture diverse in one of the five boroughs of New York City with it having the largest borough geographically.

## Solving the Problem Using Data Science
We are going to take advantage of the Foursquare database to gain insight about the New York area, eventual competitors, and optimal location.

## Data section
We will import the ZIP Code Definitions of New York City Neighborhoods which is available from https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm and append them to our dataset, in order to pinpoint the optimal location, scoping our way through a multitude of venues thanks to the Foursquare API.

## Methodology

Methodology section

    Collect Data
    Explore and Understand Data
    Data Preparation and Preprocessing
    Modeling



#### Collect Data

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import json 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.86 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.63 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.61 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.19 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |###################

In [3]:
link = "https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm"
df = pd.read_html(link,header=0)[0]
url = "https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm"
wiki = requests.get(url)
bs = BeautifulSoup(wiki.content, "html.parser")
table = bs.find_all("table")[0]
table_rows = table.find_all("tr")

In [4]:
#reading into list of columns

c1=[]
c2=[]
c3=[]


for tr in table_rows:
    #header = tr.find_all("th")
    row = tr.find_all("td")
    if len(row) == 3:
        c1.append(row[0].find(text = True))
        c2.append(row[1].find(text = True))
        c3.append(row[2].find(text = True))

In [5]:
#reading into Data Frame

df = pd.DataFrame(c1, columns = ["Postcode"])
df["Borough"] = c2
df["Neighborhood"] = c3

In [6]:
df.head()

,Postcode,Borough,Neighborhood
0,Bronx,Central Bronx,"10453, 10457, 10460"
1,Brooklyn,Central Brooklyn,"11212, 11213, 11216, 11233, 11238"
2,Manhattan,Central Harlem,"10026, 10027, 10030, 10037, 10039"
3,Queens,Northeast Queens,"11361, 11362, 11363, 11364"
4,Staten Island,Port Richmond,"10302, 10303, 10310"


In [7]:
df.to_csv("NYZipCodes.csv")

In [8]:
df= pd.read_csv("NYZipCodes.csv", index_col=0)

df.head()

,Postcode,Borough,Neighborhood
0,Bronx,Central Bronx,"10453, 10457, 10460"
1,Brooklyn,Central Brooklyn,"11212, 11213, 11216, 11233, 11238"
2,Manhattan,Central Harlem,"10026, 10027, 10030, 10037, 10039"
3,Queens,Northeast Queens,"11361, 11362, 11363, 11364"
4,Staten Island,Port Richmond,"10302, 10303, 10310"


In [16]:
final = final[['Borough','Neighborhood','Latitude','Longitude']]
final.head()

,Borough,Neighborhood,Latitude,Longitude
Index,,,,
1,Bronx,Central Bronx,40.852779,-73.912332
2,Bronx,Bronx Park and Fordham,40.862543,-73.888143
3,Bronx,High Bridge and Morrisania,40.820479,-73.925084
4,Bronx,Hunts Point and Mott Haven,40.805489,-73.916585
5,Bronx,Kingsbridge and Riverdale,40.880678,-73.906540


In [17]:
address = 'New York, NY'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York are 40.7308619, -73.9871558.


In [18]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final['Latitude'], final['Longitude'], final['Borough'], final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [19]:
newyork_data = final[final['Borough'].str.contains('Queens', regex=True)]
# Dataframe where the Borough name contains "Brooklyn" 
downtown_newyork = newyork_data[newyork_data['Borough'] == 'Queens'].reset_index(drop=True)

In [21]:
address = 'New York, NY'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Manhattan are 40.7308619, -73.9871558.


In [22]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_newyork['Latitude'], downtown_newyork['Longitude'], downtown_newyork['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [23]:
# The code was removed by Watson Studio for sharing.

Your Foursquare credentials have been saved into CLIENT_ID and CLIENT_SECRET, version has been set to 20180605


In [24]:
downtown_newyork.loc[0, 'Neighborhood']

'Northeast Queens'

In [25]:
# neighborhood latitude value
neighborhood_latitude = downtown_newyork.loc[0, 'Latitude']
# neighborhood longitude value
neighborhood_longitude = downtown_newyork.loc[0, 'Longitude']
# neighborhood name
neighborhood_name = downtown_newyork.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Northeast Queens are 40.764191, -73.772775.


In [28]:
LIMIT = 100
radius = 500
#create Foursquare API url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [29]:
# Send the GET request and examine the resutls
results = requests.get(url).json()

In [30]:
# Now extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The French Workshop,Bakery,40.765404,-73.771861
1,Martha's Country Bakery,Bakery,40.763422,-73.770971
2,Press 195,Bar,40.763905,-73.770946
3,Avli Little Greek Tavern,Greek Restaurant,40.765729,-73.771972
4,Nippon Cha,Noodle House,40.764408,-73.771461


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

77 venues were returned by Foursquare.


In [33]:
# We create a function to repeat the same process to all the neighborhoods in downtown_newyork
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
downtown_newyork_venues = getNearbyVenues(names=downtown_newyork['Neighborhood'],
                                   latitudes=downtown_newyork['Latitude'],
                                   longitudes=downtown_newyork['Longitude']
                                  )

Northeast Queens
North Queens
Central Queens
Jamaica
Northwest Queens
West Central Queens
Rockaways
Southeast Queens
Southwest Queens
West Queens


In [35]:
print(downtown_newyork_venues.shape)
downtown_newyork_venues.head()

(317, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northeast Queens,40.764191,-73.772775,The French Workshop,40.765404,-73.771861,Bakery
1,Northeast Queens,40.764191,-73.772775,Martha's Country Bakery,40.763422,-73.770971,Bakery
2,Northeast Queens,40.764191,-73.772775,Press 195,40.763905,-73.770946,Bar
3,Northeast Queens,40.764191,-73.772775,Avli Little Greek Tavern,40.765729,-73.771972,Greek Restaurant
4,Northeast Queens,40.764191,-73.772775,Nippon Cha,40.764408,-73.771461,Noodle House


In [36]:
downtown_newyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Queens,24,24,24,24,24,24
Jamaica,9,9,9,9,9,9
North Queens,17,17,17,17,17,17
Northeast Queens,77,77,77,77,77,77
Northwest Queens,64,64,64,64,64,64
Rockaways,5,5,5,5,5,5
Southeast Queens,26,26,26,26,26,26
Southwest Queens,23,23,23,23,23,23
West Central Queens,39,39,39,39,39,39


In [37]:
print('There are {} uniques categories.'.format(len(downtown_newyork_venues['Venue Category'].unique())))

There are 118 uniques categories.


In [39]:
# Analyze Each Neighborhood
# Perform one hot encoding
downtown_onehot = pd.get_dummies(downtown_newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,Bagel Shop,Bakery,...,Toy / Game Store,Trail,Train,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Northeast Queens,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Northeast Queens,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Northeast Queens,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Northeast Queens,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Northeast Queens,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Accessories Store,American Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,Bagel Shop,Bakery,...,Toy / Game Store,Trail,Train,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Central Queens,0.041667,0.000000,0.000000,0.000000,0.041667,0.041667,0.041667,0.041667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Jamaica,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000
2,North Queens,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,...,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Northeast Queens,0.000000,0.038961,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987,0.025974,...,0.000000,0.012987,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987,0.000000,0.012987
4,Northwest Queens,0.000000,0.031250,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.015625,0.000000,0.015625,0.000000
5,Rockaways,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Southeast Queens,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Southwest Queens,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Central Queens,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.025641,...,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000
9,West Queens,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [41]:
# print each neighborhood's top 5 most common venues
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Queens----
                venue  freq
0  Chinese Restaurant  0.12
1         Pizza Place  0.08
2         Bus Station  0.08
3   Accessories Store  0.04
4      Ice Cream Shop  0.04


----Jamaica----
                  venue  freq
0    Chinese Restaurant  0.11
1    Seafood Restaurant  0.11
2              Pharmacy  0.11
3      Basketball Court  0.11
4  Caribbean Restaurant  0.11


----North Queens----
               venue  freq
0        Supermarket  0.18
1  Korean Restaurant  0.18
2              Train  0.06
3     Cosmetics Shop  0.06
4          Pool Hall  0.06


----Northeast Queens----
               venue  freq
0                Bar  0.08
1        Pizza Place  0.05
2                Spa  0.04
3                Pub  0.04
4  Indian Restaurant  0.04


----Northwest Queens----
                venue  freq
0                Café  0.11
1               Hotel  0.08
2         Coffee Shop  0.08
3  Mexican Restaurant  0.06
4       Deli / Bodega  0.06


----Rockaways----
                venue 

In [43]:
# feed into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Queens,Chinese Restaurant,Pizza Place,Bus Station,Accessories Store,Bubble Tea Shop
1,Jamaica,Seafood Restaurant,Deli / Bodega,Chinese Restaurant,Video Store,Caribbean Restaurant
2,North Queens,Korean Restaurant,Supermarket,Pool Hall,Toy / Game Store,Martial Arts Dojo
3,Northeast Queens,Bar,Pizza Place,Indian Restaurant,Pub,Sushi Restaurant
4,Northwest Queens,Café,Hotel,Coffee Shop,Deli / Bodega,Mexican Restaurant


In [45]:
# Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 2, 1, 1, 0, 1, 1, 1, 4], dtype=int32)

In [46]:
downtown_data = downtown_newyork
downtown_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Northeast Queens,40.764191,-73.772775
1,Queens,North Queens,40.768208,-73.827403
2,Queens,Central Queens,40.739634,-73.794490
3,Queens,Jamaica,40.698095,-73.758986
4,Queens,Northwest Queens,40.747155,-73.939750


In [47]:
downtown_data.shape

(10, 4)

In [48]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_merged = downtown_data

# add clustering labels
downtown_merged['Cluster Labels'] = kmeans.labels_

# merge newyork_grouped with newyork_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Queens,Northeast Queens,40.764191,-73.772775,1,Bar,Pizza Place,Indian Restaurant,Pub,Sushi Restaurant
1,Queens,North Queens,40.768208,-73.827403,3,Korean Restaurant,Supermarket,Pool Hall,Toy / Game Store,Martial Arts Dojo
2,Queens,Central Queens,40.739634,-73.794490,2,Chinese Restaurant,Pizza Place,Bus Station,Accessories Store,Bubble Tea Shop
3,Queens,Jamaica,40.698095,-73.758986,1,Seafood Restaurant,Deli / Bodega,Chinese Restaurant,Video Store,Caribbean Restaurant
4,Queens,Northwest Queens,40.747155,-73.939750,1,Café,Hotel,Coffee Shop,Deli / Bodega,Mexican Restaurant


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,West Central Queens,Grocery Store,Chinese Restaurant,Sushi Restaurant,Japanese Restaurant,Gym


In [51]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Northeast Queens,Bar,Pizza Place,Indian Restaurant,Pub,Sushi Restaurant
3,Jamaica,Seafood Restaurant,Deli / Bodega,Chinese Restaurant,Video Store,Caribbean Restaurant
4,Northwest Queens,Café,Hotel,Coffee Shop,Deli / Bodega,Mexican Restaurant
6,Rockaways,Metro Station,Deli / Bodega,Chinese Restaurant,Liquor Store,Beach
7,Southeast Queens,Pharmacy,Gift Shop,Pet Store,Pizza Place,Cosmetics Shop
8,Southwest Queens,Italian Restaurant,Clothing Store,Fast Food Restaurant,Spa,Mexican Restaurant


In [52]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Central Queens,Chinese Restaurant,Pizza Place,Bus Station,Accessories Store,Bubble Tea Shop


In [53]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,North Queens,Korean Restaurant,Supermarket,Pool Hall,Toy / Game Store,Martial Arts Dojo


In [54]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,West Queens,Tennis Stadium,Deli / Bodega,Pizza Place,Hotel,Mexican Restaurant


# Results and Discussion

All clusters have 20 restaurants and food places on the area that caters to specific cuisines.    
Cluster 1 has 1 neighborhood with 3 restaurants as common venues, 1 store, and a gym.    
Cluster 2 has 6 neighborhoods with 13 distinct variety of restaurants but has different types of stores.    
Cluster 3 has 1 neighborhood with 2 restaurants and a specialized bar.    
Cluster 4 has 1 neighborhood with 1 restaurant, a supermarket, and a bus station.        
Cluster 5 has 1 neighborhood with 2 restaurants as common venues, a hotel and a tennis stadium.    
The geographic location does give us a good overview of the metropolitan environment. A good approach for a new businesses might be to start where there's a high concentration of restaurants, and cater to one of the specific cultures found in the borough.
The neighbourhood from Cluster 2 appears to be the best candidate for our purpose.


# Conclusion

The KMeans method we applied gives a new entrepreneur a good starting point on what business model is more likely to develop successfully, based on the type of venues identified using Foursquare api.
The cluster of venues identified within the neighborhood is predominantly restaurants and food shops, but there is no nearby Korean restaurant or bubble tea shop - two kind of venue that appear to be trending in other boroughs - both excellent choices for a new, trendy activity.

